## Benchmarking

In a distributed environment, the same computation can have different runtimes, depending on the size of the cluster, partitioning of the data, computational parallelism, and other factors.

This notebook demonstrates how turning different knobs impacts computation time.  Here are the main takeaways:

* Dask is a lot faster when data is spread across multiple partitions so the computations can be run in parallel
* Persisting DataFrames in memory can be a great performance optimization
* Figuring out the optimal partitioning for a given computation is challenging
* Lazy execution also makes benchmarking challenging

### Setup Coiled cluster for Dask computation environment

In [112]:
import coiled
from dask.distributed import Client
import dask.dataframe as dd
import time

In [113]:
import warnings
warnings.filterwarnings('ignore')

In [114]:
cluster = coiled.Cluster(name="benchmarking", n_workers=10)

Found software environment build


In [115]:
client = Client(cluster)

## Count benchmarking

This section reads a month of the NYC taxi data into different Dask DataFrames, performs a count, and measures computation time.  Here are the scenarios examined:

1. Persisted DataFrame with 41 partitions
2. Unpersisted DataFrame with 41 partitions
3. Unpersisted DataFrame with 11 partitions
4. Unpersisted DataFrame with 1 partition

DataFrames with multiple partitions can perform count computations in parallel and that's why they're much faster!  Let's quantify the speed gains Dask provides.

In [116]:
dtype = {
    "payment_type": "UInt8",
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",
}
path = "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv"

### Create Persisted DataFrame

In [117]:
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

In [118]:
df.npartitions

41

### Create unpersisted DataFrame

In [119]:
df_unpersisted = dd.read_csv(
    path,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
    blocksize="16 MiB",
)

In [120]:
df_unpersisted.npartitions

41

### Create unpersisted DataFrame without setting blocksize

In [121]:
df_no_blocksize = dd.read_csv(
    path,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
)

In [122]:
df_no_blocksize.npartitions

11

### Create unpersisted DataFrame with blocksize set to None

In [123]:
df_none_blocksize = dd.read_csv(
    path,
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype=dtype,
    storage_options={"anon": True},
    blocksize=None,
)

In [124]:
df_none_blocksize.npartitions

1

### Define some benchmark helper functions

In [86]:
dask_benchmarks = {
    'duration': [],  # in seconds
    'task': [],
}

In [87]:
def benchmark(f, df, benchmarks, name, **kwargs):
    start_time = time.time()
    ret = f(df, **kwargs)
    benchmarks['duration'].append(time.time() - start_time)
    benchmarks['task'].append(name)
    print(f"{name} took: {benchmarks['duration'][-1]} seconds")
    return benchmarks['duration'][-1]

### Run the count benchmarks with the different DataFrames

In [88]:
def count(df):
    return len(df)

In [89]:
benchmark(count, df=df, benchmarks=dask_benchmarks, name='count_persisted')

count_persisted took: 0.15817594528198242 seconds


0.15817594528198242

In [90]:
benchmark(count, df=df_unpersisted, benchmarks=dask_benchmarks, name='count_unpersisted')

count_unpersisted took: 5.21872615814209 seconds


5.21872615814209

In [91]:
benchmark(count, df=df_no_blocksize, benchmarks=dask_benchmarks, name='count_no_blocksize')

count_no_blocksize took: 5.67988395690918 seconds


5.67988395690918

In [92]:
benchmark(count, df=df_none_blocksize, benchmarks=dask_benchmarks, name='count_none_blocksize')

count_none_blocksize took: 33.84476184844971 seconds


33.84476184844971

## Count benchmarking conclusions

Running the count operation on a persisted DataFrame with 41 partitions is by far the fastest.  The operation is much slower when the DataFrame isn't persisted and when fewer partitions are used.  Using one partition is particularily slow and 

## mean benchmarking

Let's calculate the mean fare using the same DataFrames as above and see if the computation time results are similar.

In [94]:
def mean(df):
    return df.fare_amount.mean().compute()

In [95]:
benchmark(mean, df=df, benchmarks=dask_benchmarks, name='mean_persisted')

mean_persisted took: 0.2158820629119873 seconds


0.2158820629119873

In [96]:
benchmark(mean, df=df_unpersisted, benchmarks=dask_benchmarks, name='mean_unpersisted')

mean_unpersisted took: 5.3857622146606445 seconds


5.3857622146606445

In [97]:
benchmark(mean, df=df_no_blocksize, benchmarks=dask_benchmarks, name='mean_no_blocksize')

mean_no_blocksize took: 5.542102098464966 seconds


5.542102098464966

In [98]:
benchmark(mean, df=df_none_blocksize, benchmarks=dask_benchmarks, name='mean_none_blocksize')

mean_none_blocksize took: 36.455934047698975 seconds


36.455934047698975

## Group by benchmarking

In [135]:
start_time = time.time()
df.groupby('pickup_day').fare_amount.sum().compute()
time.time() - start_time

0.6893270015716553

In [136]:
def light_benchmark(f):
  start_time = time.time()
  f()
  return time.time() - start_time

In [143]:
light_benchmark(lambda: df.groupby('pickup_day').fare_amount.sum().compute())

0.6474931240081787

In [139]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_day
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,2019-01-01
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,2019-01-01
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,2018-12-21
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,2018-11-28
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,2018-11-28


In [140]:
df_unpersisted["pickup_day"] = df_unpersisted["tpep_pickup_datetime"].dt.date

In [142]:
df_unpersisted.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_day
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,2019-01-01
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,2019-01-01
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,2018-12-21
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,2018-11-28
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,2018-11-28


In [144]:
light_benchmark(lambda: df_unpersisted.groupby('pickup_day').fare_amount.sum().compute())

6.200210094451904

In [145]:
df_unpersisted2 = df_unpersisted.set_index("pickup_day")

In [146]:
df_unpersisted2.npartitions

41

In [148]:
light_benchmark(lambda: df_unpersisted2.groupby('pickup_day').fare_amount.sum().compute())

7.9316301345825195